In [1]:
import os
os.chdir('/data/xiaodan8/research/self-attentive-parser/src')##to this folder当前的文件夹

path='/data/xiaodan8/research/dataset/FineGym'##where data is.存data的文件夹
import treesvideo
import dataloaderorg
import numpy as np
import vocabulary

#import pyximport
#pyximport.install(setup_args={"include_dirs": np.get_include()})
import chart_helper

/data/xiaodan8/anaconda3/envs/parsing/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loaders = dataloaderorg.construct_dataloaders(path, path+'/I3D_features', 32, 0, 44739242,1,5,5,20)

In [3]:
trees=treesvideo.load_trees_alt(mode='test',path=path)
testptree=[]
for tree in trees:
    testptree.append(tree.convert())

In [4]:
trees=treesvideo.load_trees_alt(mode='train',path=path)
trainptree=[]
for tree in trees:
    trainptree.append(tree.convert())

In [5]:
def vocab_helper(tree):
    labels=[]
    if type(tree)==treesvideo.InternalParseNode:
        labels.append(tree.label)
        if len(tree.children)>0:
            for child in tree.children:
                labels=labels+vocab_helper(child)
    return labels

    

In [6]:
labels=[]
for tree in trainptree:
    labels=labels+vocab_helper(tree)
for tree in testptree:
    labels=labels+vocab_helper(tree)
node_vocab=vocabulary.Vocabulary()
node_vocab.index(())
for label in labels:
    node_vocab.index(label)
node_vocab.freeze()

In [15]:
dir(tree)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'children',
 'convert',
 'enclosing',
 'label',
 'leaves',
 'left',
 'nocache',
 'oracle_label',
 'oracle_splits',
 'right']

In [7]:
'''
label_scores_chart=np.zeros((trainptree[1442].right+1,trainptree[1442].right+1,node_vocab.size))
label_scores_chart[0,10,node_vocab.index(('UNK',))]=102
label_scores_chart[0,3,node_vocab.index(('BB_flight_handspring', '335'))]=101
label_scores_chart[3,6,node_vocab.index(('BB_flight_salto', '295'))]=100
label_scores_chart[6,10,node_vocab.index(('BB_turns', '272'))]=99
label_scores_chart[0,1,node_vocab.index(())]=50
label_scores_chart[1,2,node_vocab.index(())]=50
label_scores_chart[2,3,node_vocab.index(())]=50
label_scores_chart[3,4,node_vocab.index(())]=50
label_scores_chart[4,5,node_vocab.index(())]=50
label_scores_chart[5,6,node_vocab.index(())]=50
label_scores_chart[6,7,node_vocab.index(())]=50
label_scores_chart[7,8,node_vocab.index(())]=50
label_scores_chart[8,9,node_vocab.index(())]=50
label_scores_chart[9,10,node_vocab.index(())]=50
'''

In [29]:
from numpy import random
label_scores_chart=random.randint(0,100,size=(trainptree[1442].right+1,trainptree[1442].right+1,node_vocab.size))
for i in range(trainptree[1442].right+1):
    for j in range(trainptree[1442].right+1):
        label_scores_chart[i,j,0]+=random.randint(50)


In [37]:
#import importlib
#import chart_helper #import the module here, so that it can be reloaded.
#importlib.reload(chart_helper)

#test mode
score, p_i, p_j, p_label, _ = chart_helper.decode(False, trainptree[1442].right, label_scores_chart, False, None, node_vocab)

In [31]:
#force gold mode
#chart_helper.decode(True, trainptree[1442].right, label_scores_chart, False, trainptree[1442], node_vocab)

(1299.0,
 [0, 0, 0, 1, 1, 2, 3, 3, 3, 4, 4, 5, 6, 6, 7, 7, 8, 8, 9],
 [10, 3, 1, 3, 2, 3, 10, 6, 4, 6, 5, 6, 10, 7, 10, 8, 10, 9, 10],
 [4, 11, 0, 0, 0, 0, 0, 27, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0],
 0)

In [32]:
#train mode
#chart_helper.decode(False, trainptree[1442].right, label_scores_chart, True, trainptree[1442], node_vocab)

(2025.0,
 [0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, 9],
 [10, 8, 1, 8, 2, 8, 3, 8, 4, 8, 5, 8, 7, 6, 7, 8, 10, 9, 10],
 [14, 0, 122, 28, 0, 60, 55, 30, 62, 0, 139, 54, 0, 27, 7, 0, 0, 108, 11],
 13)

In [73]:
aim=loaders['train'].dataset[1442]
frames=[]
for i in range(len(aim[2][0])):
    for j in range(len(aim[2][0,i])):
        for k in range(len(aim[2][0,i,j])):
            if(aim[2][0,i,j,k]==0):
                break
            frames.append((loaders['train'].dataset.id2actionlabel[aim[1]['action'][0,i,j]],aim[0][0,i,j,k]))

In [72]:
#loaders['train'].dataset.id2actionlabel[1]

' 19'

In [74]:
    def decode_from_chart(p_i, p_j, p_label,node_vocab,frames):
        idx=-1
        def make_tree():
            nonlocal idx
            idx += 1
            i, j, label_idx = p_i[idx], p_j[idx], p_label[idx]
            label = node_vocab.value(label_idx)
            #if label_idx==0:
            #    label = ('-NONE-',)
            if (i + 1) >= j:
                tag, I3D = frames[i]
                tree = treesvideo.LeafParseNode_alt(int(i), tag, I3D)
                if label!=():
                    tree = treesvideo.InternalParseNode(label, [tree])
                return [tree]
            else:
                left_trees = make_tree()
                right_trees = make_tree()
                children = left_trees + right_trees
                if label!=():
                    return [treesvideo.InternalParseNode(label, children)]
                else:
                    return children
        return make_tree()[0]

In [76]:
exp=decode_from_chart(p_i, p_j, p_label,node_vocab,frames)

In [123]:
exp.children[1].children[1].children[1].children[2].children

(<treesvideo.InternalParseNode at 0x7fa8ee15e290>,
 <treesvideo.LeafParseNode_alt at 0x7fa8ee15e390>)

In [121]:
node_vocab.value(54)

('BB_flight_handspring', '336')

In [7]:
trees[0].children[0].label

'UB_circles'

In [8]:
trees[0].label

'UB'